In [34]:
dir = 'data/full data/'

import json
import os
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

from utils import elem2sent

import warnings
warnings.filterwarnings('ignore')

In [35]:
file = 'generated_data/similar_sentences.xlsx'
df = pd.read_excel(file)

In [36]:
df.label.value_counts()

STYLYSTIC     371
IRRELEVANT    171
RELEVANT       90
Name: label, dtype: int64

In [37]:
df_stage2 = df[['label', 'sentence1', 'sentence2']]
label2int = {'RELEVANT': 1, 'STYLYSTIC': 0, 'IRRELEVANT': -1}
df_stage2.label = df_stage2.label.map(label2int)

#train test split
train_df, test_df = train_test_split(df_stage2, test_size=0.2, random_state=42)

In [38]:
train_df.label.value_counts()

 0    296
-1    136
 1     73
Name: label, dtype: int64

In [39]:
test_df.label.value_counts()

 0    75
-1    35
 1    17
Name: label, dtype: int64

In [40]:
from autogluon.text import TextPredictor

baseline_predictor = TextPredictor(
    label='label', 
    problem_type='classification',
    eval_metric='accuracy', 
    path='./models_diffrence_detector'
)
baseline_predictor.fit(train_df, time_limit=None, presets='medium_quality_faster_train')

Global seed set to 123
Auto select gpus: [0]
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                         | Params
-------------------------------------------------------------------
0 | model             | HFAutoModelForTextPrediction | 13.5 M
1 | validation_metric | Accuracy                     | 0     
2 | loss_func         | CrossEntropyLoss             | 0     
-------------------------------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
26.968    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001F538AFE8B0>
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\legal\lib\site-packages\torch\utils\data\dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "c:\ProgramData\Anaconda3\envs\legal\lib\site-packages\torch\utils\data\dataloader.py", line 1468, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [26]:
test_score = baseline_predictor.evaluate(test_df, metrics=['acc'])
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.9024390243902439}


In [29]:
test_df.iloc[[0]]

,label,sentence1,sentence2
70,0,earnings of nationals or residents of the othe...,the earnings of nationals of the other contrac...


In [30]:
#inferece on the first row of test_df
baseline_predictor.predict(test_df.iloc[0:1, 1:3])

Predicting: 0it [00:00, ?it/s]

70    0
Name: label, dtype: int64

In [43]:
test_df.iloc[0:1, 1:3]

,sentence1,sentence2
95,returns means the amounts after payment of tax...,the term returns means the amounts yielded by...


In [33]:
test_df.head()

,Label,sentence1,sentence2
95,0,returns means the amounts after payment of tax...,the term returns means the amounts yielded by...
15,1,business concessions conferred by law by admin...,rights to engage in economic activities confe...
30,0,investors of either contracting party whose in...,without prejudice to the provisions of article...
159,1,where the measures are accompanied by provisio...,the measures are accompanied by provisions for...
186,1,investors of one contracting party whose inves...,investments of either contracting state or any...


In [48]:
print(test_df.iloc[0:1, 1:3].sentence1.values[0], '\n', test_df.iloc[0:1, 1:3].sentence2.values[0])

returns means the amounts after payment of tax yielded by such investments and in particular though not exclusively includes profits interest capital gains dividends royalties or fees 
  the term returns means the amounts yielded by an investment and includes in particular though not exclusively profit interest capital gains royalties fees and dividends


In [14]:
#vocab = baseline_predictor._learner.get_inputs()[0].vocab

AttributeError: 'TextPredictor' object has no attribute '_learner'